In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['CUDA_DEVICE_ORDER']='PCI_BUS_ID'

In [2]:
import pandas as pd
import numpy as np
import pickle
from sklearn.metrics.pairwise import cosine_similarity
import re

# Training BERT

# Training model_1

In [4]:
df1 = pd.read_csv("data/product_df.csv")
df1.columns = [0,1,2]
df1.head(5)

,0,1,2
0,amari ireland largest independent metal stockh...,"product, motor vehicles, buses and trucks, mot...",0
1,davies chemists established john glynne davies...,"product, pharmaceutical products medicines dru...",1
2,browser doesn support javascript.,"product, ship spare parts, boats, yachts",0
3,agilent acquisition asset young scientific ins...,"product, water heaters, water purifiers, water...",1
4,load click https www. com featured_page antenn...,"product, electrical and electronic appliances,...",0


In [5]:
fold = int(len(df1)*0.95)
train_1 = df1
val_1 = df1[fold:]

### Making STS-like dataset

In [6]:
from torch.utils.data import DataLoader
import math
from sentence_transformers import models, losses
from sentence_transformers import SentencesDataset, LoggingHandler, SentenceTransformer
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from sentence_transformers.readers import *
import logging
from datetime import datetime


import csv
import gzip
import os


##### Custom reader
class DataReader:
    """
    Reads in the STS dataset. Each line contains two sentences (s1_col_idx, s2_col_idx) and one label (score_col_idx)
    """
    def __init__(self, dataset_folder, s1_col_idx=0, s2_col_idx=1, score_col_idx=2, delimiter="\t",
                 quoting=csv.QUOTE_NONE, normalize_scores=False, min_score=0, max_score=1):
        self.dataset_folder = dataset_folder
        self.score_col_idx = score_col_idx
        self.s1_col_idx = s1_col_idx
        self.s2_col_idx = s2_col_idx
        self.delimiter = delimiter
        self.quoting = quoting
        self.normalize_scores = normalize_scores
        self.min_score = min_score
        self.max_score = max_score

    def get_examples(self, df, modelname = 'model_1', max_examples=0):
        """
        filename specified which data split to use (train.csv, dev.csv, test.csv).
        """
        examples = []
        for r in df.iterrows():
            id = r[0]
            row = r[1]
            score = float(row[self.score_col_idx])

            s1 = row[self.s1_col_idx]
            s2 = row[self.s2_col_idx]
            examples.append(InputExample(guid=modelname+str(id), texts=[s1, s2], label=score))

        return examples

In [7]:
# Read the dataset
model_name = 'roberta-base'
# 'bert-base-nli-stsb-mean-tokens'

batch_size = 16
reader = DataReader('')
model_save_path = ' '+model_name+'-'+datetime.now().strftime("%Y-%m-%d_%H-%M-%S")


# Use RoBERTa for mapping tokens to embeddings
word_embedding_model = models.RoBERTa(model_name)

# Apply mean pooling to get one fixed sized sentence vector
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(),
                               pooling_mode_mean_tokens=True,
                               pooling_mode_cls_token=False,
                               pooling_mode_max_tokens=False)

model = SentenceTransformer(modules=[word_embedding_model, pooling_model])


model1 = SentenceTransformer('bert-base-nli-mean-tokens')

In [8]:
# training data
train_data = SentencesDataset(reader.get_examples(train_1, modelname = 'model_1'), model=model1)
train_dataloader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
train_loss = losses.CosineSimilarityLoss(model=model1)

print('done')

# val data
dev_data = SentencesDataset(reader.get_examples(val_1, modelname = 'model_1'), model=model1)
dev_dataloader = DataLoader(dev_data, shuffle=False, batch_size=batch_size)
evaluator = EmbeddingSimilarityEvaluator(dev_dataloader)

done


## Fitting the model

In [9]:
# Configure the training
num_epochs = 1

warmup_steps = math.ceil(len(train_dataloader) * num_epochs / batch_size * 0.1) #10% of train data for warm-up

# Train the model
model1.fit(train_objectives=[(train_dataloader, train_loss)],
          evaluator=evaluator,
          epochs=num_epochs,
          evaluation_steps=100,
          warmup_steps=warmup_steps,
          output_path=model_save_path
          )

Iteration:  56%|█████▌    | 256/461 [01:54<01:26,  2.37it/s]


Epoch: 100%|██████████| 1/1 [03:35<00:00, 215.11s/it]70it/s]


# Model_2 fitting

In [ ]:
df2 = pd.read_csv("data/function_df.csv")

fold = int(len(df2)*0.95)
train_2 = df2
val_2 = df2[fold:]

# Read the dataset
model_name = '2222bert-base-nli-mean-tokens'
batch_size = 16
reader = DataReader('')
model_save_path = ' '+model_name+'-'+datetime.now().strftime("%Y-%m-%d_%H-%M-%S")


model2 = SentenceTransformer('bert-base-nli-mean-tokens')

In [ ]:
# training data
train_data = SentencesDataset(reader.get_examples(train_2, modelname = 'model_2'), model=model2)
train_dataloader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
train_loss = losses.CosineSimilarityLoss(model=model2)

# val data
dev_data = SentencesDataset(reader.get_examples(val_2, modelname = 'model_2'), model=model2)
dev_dataloader = DataLoader(dev_data, shuffle=False, batch_size=batch_size)
evaluator = EmbeddingSimilarityEvaluator(dev_dataloader)

In [ ]:
# Configure the training
num_epochs = 1

warmup_steps = math.ceil(len(train_dataloader) * num_epochs / batch_size * 0.1) #10% of train data for warm-up

# Train the model
model2.fit(train_objectives=[(train_dataloader, train_loss)],
          evaluator=evaluator,
          epochs=num_epochs,
          evaluation_steps=100,
          warmup_steps=warmup_steps,
          output_path=model_save_path
          )

Iteration:  28%|██▊       | 526/1905 [05:03<10:41,  2.15it/s]


Iteration:  41%|████      | 776/1905 [07:24<08:46,  2.14it/s]


Iteration:  54%|█████▍    | 1026/1905 [10:04<06:14,  2.35it/s]


Iteration:  58%|█████▊    | 1099/1905 [10:36<06:04,  2.21it/s]

In [ ]:
%%time

with open('data/train_only_text.pickle', 'rb') as f:
    train = pickle.load(f)
    
with open('data/test_only_text.pickle', 'rb') as f:
    test = pickle.load(f)
    

train["af_embed"] = model2.encode(list(train["accepted_function"].values))
train["rf_embed"] = model2.encode(list(train["rejected_function"].values))
train["ap_embed"] = model1.encode(list(train["accepted_product"].values))
train["rp_embed"] = model1.encode(list(train["rejected_product"].values))

# train["text_embed"] = model.encode(list(train["text"].values))
train["text_embed"] = model1.encode(list(train["text"].values))
# train["text_embed"] = model.encode(list(train["text"].values))
train["text_embed_2"] = model2.encode(list(train["text"].values))


test["af_embed"] = model2.encode(list(test["accepted_function"].values))
test["rf_embed"] = model2.encode(list(test["rejected_function"].values))
test["ap_embed"] = model1.encode(list(test["accepted_product"].values))
test["rp_embed"] = model1.encode(list(test["rejected_product"].values))

# train["text_embed"] = model.encode(list(train["text"].values))
test["text_embed"] = model1.encode(list(test["text"].values))
# train["text_embed"] = model.encode(list(train["text"].values))
test["text_embed_2"] = model2.encode(list(test["text"].values))

In [ ]:
import pickle

ss = train["ap_embed"][0].shape[0]

tmp1 = pd.DataFrame().from_records(train["af_embed"])
tmp1.columns = [f'af_{str(i)}' for i in range(ss)]

tmp2 = pd.DataFrame().from_records(train["rf_embed"])
tmp2.columns = [f'rf_{str(i)}' for i in range(ss)]

tmp3 = pd.DataFrame().from_records(train["ap_embed"])
tmp3.columns = [f'ap_{str(i)}' for i in range(ss)]

tmp4 = pd.DataFrame().from_records(train["rp_embed"])
tmp4.columns = [f'rp_{str(i)}' for i in range(ss)]

tmp5 = pd.DataFrame().from_records(train["text_embed"])
tmp5.columns = [f'tt_{str(i)}' for i in range(ss)]

tmp6 = pd.DataFrame().from_records(train["text_embed_2"])
tmp6.columns = [f'tt2_{str(i)}' for i in range(ss)]


tmp = pd.concat([tmp1, tmp2, tmp3, tmp4, tmp5, tmp6], axis = 1)

tmp['id'] = train['id']
columns1 = [f'ap_{str(i)}' for i in range(ss)]+[f'rp_{str(i)}' for i in range(ss)]+[f'tt_{str(i)}' for i in range(ss)]
columns2 = [f'af_{str(i)}' for i in range(ss)]+[f'rf_{str(i)}' for i in range(ss)]+[f'tt2_{str(i)}' for i in range(ss)]

dict_to_write = {'train_x':tmp,
                 'train_y':train.target,
                 'columns_1':columns1,
                 'columns_2':columns2
}

with open('train_bert_7.pickle', 'wb') as handle:
    pickle.dump(dict_to_write, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
    
    
ss = test["ap_embed"][0].shape[0]

tmp1 = pd.DataFrame().from_records(test["af_embed"])
tmp1.columns = [f'af_{str(i)}' for i in range(ss)]

tmp2 = pd.DataFrame().from_records(test["rf_embed"])
tmp2.columns = [f'rf_{str(i)}' for i in range(ss)]

tmp3 = pd.DataFrame().from_records(test["ap_embed"])
tmp3.columns = [f'ap_{str(i)}' for i in range(ss)]

tmp4 = pd.DataFrame().from_records(test["rp_embed"])
tmp4.columns = [f'rp_{str(i)}' for i in range(ss)]

tmp5 = pd.DataFrame().from_records(test["text_embed"])
tmp5.columns = [f'tt_{str(i)}' for i in range(ss)]

tmp6 = pd.DataFrame().from_records(test["text_embed_2"])
tmp6.columns = [f'tt2_{str(i)}' for i in range(ss)]


tmp = pd.concat([tmp1, tmp2, tmp3, tmp4, tmp5, tmp6], axis = 1)

tmp['id'] = test['id']

columns1 = [f'ap_{str(i)}' for i in range(ss)]+[f'rp_{str(i)}' for i in range(ss)]+[f'tt_{str(i)}' for i in range(ss)]
columns2 = [f'af_{str(i)}' for i in range(ss)]+[f'rf_{str(i)}' for i in range(ss)]+[f'tt2_{str(i)}' for i in range(ss)]    
    

dict_to_write_test = {'test_x':tmp,
                 'columns_1':columns1,
                 'columns_2':columns2
}

    
with open('test_bert_7.pickle', 'wb') as handle:
    pickle.dump(dict_to_write_test, handle, protocol=pickle.HIGHEST_PROTOCOL)